In [0]:
from pyspark.sql.functions import to_date, year, dayofyear
from pyspark.sql.functions import when, col, count, lit, round, avg, mean, min, max, first, stddev

In [0]:
%sql

REFRESH TABLE full_player_stats_with_teams;
REFRESH TABLE team_standings;

In [0]:
table_all = sqlContext.sql("select * from `full_player_stats_with_teams` ")
table_all.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Rank: long (nullable = true)
 |-- FirstPlace: float (nullable = true)
 |-- PtsWon: float (nullable = true)
 |-- PtsMax: float (nullable = true)
 |-- %: float (nullable = true)
 |-- Pos: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- Gs: integer (nullable = true)
 |-- MP: float (nullable = true)
 |-- FG: float (nullable = true)
 |-- FGA: float (nullable = true)
 |-- FG%: float (nullable = true)
 |-- 3P: float (nullable = true)
 |-- 3PA: float (nullable = true)
 |-- 3P%: float (nullable = true)
 |-- 2P: float (nullable = true)
 |-- 2PA: float (nullable = true)
 |-- 2P%: float (nullable = true)
 |-- eFG%: float (nullable = true)
 |-- FT: float (nullable = true)
 |-- FTA: float (nullable = true)
 |-- FT%: float (nullable = true)
 |-- ORB: float (nullable = true)
 |-- DRB: float (nullable = true)
 |-- TRB: float (nullab

In [0]:
winners = table_all.filter(table_all.Rank==1)
finalist = table_all.filter((table_all.Rank<=3) & ((table_all.Rank>1)))
contenders = table_all.filter((table_all.Rank<=5) & ((table_all.Rank>3)))
table = table_all.filter(table_all.Rank <= 10)




Sajt sa objašnjenjima statistika: https://www.basketball-reference.com/about/glossary.html

In [0]:
display(table)


Team,Year,Player,Age,Rank,FirstPlace,PtsWon,PtsMax,%,Pos,G,Gs,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Rk,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post,VoteType
UTA,1999,Karl Malone,35,1,44.0,827.0,1180.0,0.701,F,49,49,37.4,8.0,16.3,0.493,0.0,0.0,0.0,8.0,16.2,0.494,0.493,7.7,9.8,0.788,2.2,7.3,9.4,4.1,1.3,0.6,3.3,2.7,23.8,25.6,0.577,0.001,0.602,7.7,22.1,15.4,23.0,1.9,1.2,13.8,30.5,6.2,3.4,9.6,0.252,5.3,1.4,6.7,4.0,2,37,13,50,0.74,13,0.846,16,0.625,Winners
MIA,1999,Alonzo Mourning,28,2,36.0,773.0,1180.0,0.655,C,46,46,38.1,7.0,13.8,0.511,0.0,0.0,0.0,7.0,13.7,0.513,0.511,6.0,9.2,0.652,3.6,7.4,11.0,1.6,0.7,3.9,3.0,3.5,20.1,24.6,0.563,0.003,0.667,12.3,22.0,17.5,8.7,1.1,7.7,14.5,27.1,4.0,3.9,7.9,0.216,2.5,2.2,4.7,3.0,5,33,17,50,0.66,13,0.692,15,0.667,Finalist
SAS,1999,Tim Duncan,22,3,30.0,740.0,1180.0,0.627,F,50,50,39.3,8.4,16.9,0.495,0.0,0.1,0.143,8.3,16.8,0.498,0.495,4.9,7.2,0.69,3.2,8.2,11.4,2.4,0.9,2.5,2.9,2.9,21.7,23.2,0.541,0.008,0.424,9.7,22.2,16.3,12.1,1.2,4.4,12.7,27.2,4.0,4.7,8.7,0.213,2.7,1.8,4.5,3.2,1,37,13,50,0.74,14,0.429,16,0.875,Finalist
PHI,1999,Allen Iverson,23,4,5.0,319.0,1180.0,0.27,G,48,48,41.5,9.1,22.0,0.412,1.2,4.1,0.291,7.9,17.9,0.44,0.439,7.4,9.9,0.751,1.4,3.5,4.9,4.6,2.3,0.1,3.5,2.0,26.8,22.2,0.508,0.188,0.449,3.8,10.1,6.9,24.2,3.0,0.3,11.7,32.6,4.3,2.9,7.2,0.173,5.1,0.6,5.7,3.9,12,28,22,50,0.56,13,0.615,16,0.438,Contenders
PHO,1999,Jason Kidd,25,5,2.0,159.0,1180.0,0.135,G,50,50,41.2,6.2,14.0,0.444,0.9,2.5,0.366,5.3,11.5,0.461,0.476,3.6,4.8,0.757,1.7,5.0,6.8,10.8,2.3,0.4,3.0,2.2,16.9,22.5,0.527,0.176,0.342,4.8,14.5,9.6,44.0,3.0,0.7,15.7,21.4,5.8,2.3,8.1,0.188,4.4,1.1,5.5,3.9,14,27,23,50,0.54,13,0.615,18,0.389,Contenders
LAL,1999,Shaquille O'Neal,26,6,1.0,89.0,1180.0,0.075,C,49,49,34.8,10.4,18.1,0.576,0.0,0.0,0.0,10.4,18.0,0.577,0.576,5.5,10.2,0.54,3.8,6.9,10.7,2.3,0.7,1.7,2.5,3.2,26.3,30.6,0.584,0.001,0.563,13.1,22.6,18.0,14.4,1.1,3.5,10.0,32.4,7.2,1.9,9.0,0.255,7.5,-0.4,7.1,3.9,9,31,19,50,0.62,15,0.6,17,0.706,Other
SAC,1999,Chris Webber,25,7,0.0,51.0,1180.0,0.043,F,42,42,40.9,9.0,18.5,0.486,0.1,0.8,0.118,8.9,17.7,0.503,0.488,1.9,4.1,0.454,3.5,9.4,13.0,4.1,1.4,2.1,3.5,3.5,20.0,19.5,0.491,0.044,0.224,9.2,24.2,16.7,17.8,1.7,3.3,14.8,24.9,1.0,2.7,3.6,0.102,0.8,1.1,1.9,1.7,15,27,23,50,0.54,13,0.462,19,0.421,Other
DET,1999,Grant Hill,26,8,0.0,39.0,1180.0,0.033,F,50,50,37.0,7.7,16.0,0.479,0.0,0.3,0.0,7.7,15.8,0.487,0.479,5.7,7.6,0.752,1.3,5.8,7.1,6.0,1.6,0.5,3.7,2.3,21.1,23.9,0.543,0.017,0.473,4.3,19.3,11.7,33.6,2.4,1.2,16.0,29.6,4.5,2.8,7.3,0.189,4.8,1.4,6.2,3.8,10,29,21,50,0.58,14,0.571,17,0.647,Other
SEA,1999,Gary Payton,30,9,0.0,35.0,1180.0,0.03,G,50,50,40.2,8.0,18.5,0.434,1.7,5.6,0.295,6.4,12.8,0.495,0.479,4.0,5.5,0.721,1.2,3.6,4.9,8.7,2.2,0.2,3.1,2.3,21.7,23.1,0.519,0.304,0.299,3.5,10.6,7.0,41.3,2.9,0.4,12.9,27.1,6.0,1.2,7.2,0.172,5.1,0.0,5.1,3.6,18,25,25,50,0.5,12,0.667,17,0.412,Other
MIN,1999,Kevin Garnett,22,10,0.0,9.0,1180.0,0.008,F,47,47,37.9,8.8,19.1,0.46,0.1,0.3,0.286,8.7,18.9,0.463,0.462,3.1,4.4,0.704,3.5,6.9,10.4,4.3,1.7,1.8,2.9,3.2,20.8,22.4,0.493,0.016,0.229,9.7,22.0,15.4,21.3,2.3,3.5,12.0,27.8,2.4,3.0,5.4,0.146,3.6,1.7,5.3,3.3,17,25,25,50,0.5,13,0.615,17,0.529,Other


## Prikaz podataka

In [0]:
# Pokazuje igrače po pozicijam po godinam, ako se oni nalaze u Top 5 po broju glasova za nagradu
# Interesantno je da u 2010 među najboljim igračima nije bilo puno Centara
display(table.filter(table.Rank <= 5).groupBy("Pos","Year").count())


Pos,Year,count
G,1999,2
F,2007,2
F,1999,2
G,1997,2
F,1997,3
C,1999,1
G,1991,2
F,1991,2
C,2007,1
C,1991,1


In [0]:
# Koliko rang tima utiče na rang igrača za nagradu
display(table.groupBy("Rank").mean("Rk").orderBy("Rank"))

Rank,avg(Rk)
1,2.0
2,4.125
3,5.65625
4,7.03125
5,7.4375
6,7.0625
7,6.78125
8,7.5625
9,9.1875
10,7.09375


In [0]:
# Prikazuje koliko PER utiče na rank igrača
display(table.groupBy("Rank").mean("PER").orderBy("Rank"))

Rank,avg(PER)
1,28.209375023841858
2,26.71562498807907
3,26.296875059604645
4,25.712499976158142
5,24.515624940395355
6,23.956250071525574
7,23.487500071525574
8,22.075000166893005
9,22.481249928474426
10,22.665624916553497


In [0]:
# Prikazuje koliko USG% utiče na rank igrača
display(table.groupBy("Rank").agg(mean("USG%")).orderBy("Rank"))

Rank,avg(USG%)
1,31.16874998807907
2,29.696874916553497
3,30.29374998807907
4,30.546875059604645
5,29.068750023841858
6,28.250000298023224
7,27.446875005960464
8,27.034374982118607
9,27.006249845027924
10,26.212499856948853


In [0]:
# Prikazuje kakav je raspored poena po igračima
display(table.groupBy("VoteType","Year").agg(mean("%")).orderBy("Year"))

VoteType,Year,avg(%)
Winners,1990,0.6909999847412109
Contenders,1990,0.2044999971985817
Finalist,1990,0.6399999856948853
Other,1990,0.041800000332295896
Finalist,1991,0.5069999992847443
Contenders,1991,0.18950000405311584
Winners,1991,0.9279999732971191
Other,1991,0.03860000036656856
Finalist,1992,0.4675000011920929
Winners,1992,0.9380000233650208


In [0]:
# Broj osvojenih nagrada po poziciji igrača
table.filter(table.Rank == 1).groupBy("Pos").count().withColumnRenamed("count","NoWins").display()

Pos,NoWins
F,14
G,14
C,4


In [0]:
prime_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(PtsWon) PtsWon from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.PtsWon = b.PtsWon order by a.Age')
display(prime_table.groupBy("Age").count().orderBy("Age"))

Age,count
20,1
21,2
22,5
23,6
24,14
25,18
26,15
27,17
28,17
29,16


In [0]:
# Prikaz specifinih igrača sa njihovim brojem pojavljivanj među najboljim igračima sa obeleženim vrstom ranka   
player_rank = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, min(Rank) minRank from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and b.minRank <= 3 order by a.Age')
display(player_rank)

Team,Year,Player,Age,Rank,FirstPlace,PtsWon,PtsMax,%,Pos,G,Gs,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Rk,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post,VoteType,Player,minRank
CLE,2004,LeBron James,19,9,0.0,11.0,1230.0,0.009,G,79,79,39.5,7.9,18.9,0.417,0.8,2.7,0.29,7.1,16.1,0.438,0.438,4.4,5.8,0.754,1.3,4.2,5.5,5.9,1.6,0.7,3.5,1.9,20.9,18.3,0.488,0.145,0.308,3.5,11.8,7.6,27.8,2.2,1.3,13.9,28.2,2.4,2.6,5.1,0.078,2.3,-0.6,1.7,2.9,21,35,47,82,0.427,12,0.667,36,0.306,Other,LeBron James,1
DEN,2004,Carmelo Anthony,19,14,0.0,1.0,1230.0,0.001,F,82,82,36.5,7.6,17.9,0.426,0.8,2.6,0.322,6.8,15.3,0.444,0.449,5.0,6.4,0.777,2.2,3.8,6.1,2.8,1.2,0.5,3.0,2.7,21.0,17.6,0.509,0.146,0.358,6.8,12.1,9.4,13.8,1.7,1.0,12.7,28.5,3.7,2.4,6.1,0.098,1.6,-1.6,0.1,1.6,11,43,39,82,0.524,12,0.25,42,0.548,Other,Carmelo Anthony,3
ORL,1993,Shaquille O'Neal,20,7,0.0,30.0,980.0,0.031,C,81,81,37.9,9.0,16.1,0.562,0.0,0.0,0.0,9.0,16.1,0.563,0.562,5.3,8.9,0.592,4.2,9.6,13.9,1.9,0.7,3.5,3.8,4.0,23.4,22.9,0.584,0.002,0.553,13.4,27.1,20.6,8.5,1.0,5.7,15.9,27.0,4.5,6.0,10.4,0.163,2.0,1.5,3.5,4.3,16,41,41,82,0.5,16,0.438,48,0.563,Other,Shaquille O'Neal,1
CLE,2005,LeBron James,20,6,0.0,93.0,1270.0,0.073,F,80,80,42.4,9.9,21.1,0.472,1.4,3.9,0.351,8.6,17.2,0.499,0.504,6.0,8.0,0.75,1.4,6.0,7.4,7.2,2.2,0.7,3.3,1.8,27.2,25.7,0.554,0.183,0.378,3.8,17.0,10.2,32.9,2.8,1.1,11.8,29.7,9.7,4.6,14.3,0.203,7.0,1.7,8.6,9.1,18,42,40,82,0.512,51,0.588,31,0.387,Other,LeBron James,1
DAL,1995,Jason Kidd,21,13,0.0,7.0,1050.0,0.007,G,79,79,33.8,4.2,10.8,0.385,0.9,3.3,0.272,3.3,7.6,0.433,0.426,2.4,3.5,0.698,1.9,3.5,5.4,7.7,1.9,0.3,3.2,1.8,11.7,15.1,0.471,0.3,0.321,5.8,11.9,8.7,33.2,2.9,0.6,20.4,19.0,1.1,2.6,3.7,0.066,0.6,0.6,1.2,2.1,17,36,46,82,0.439,19,0.526,39,0.333,Other,Jason Kidd,2
OKC,2010,Kevin Durant,21,2,4.0,609.0,1230.0,0.495,F,82,82,39.5,9.7,20.3,0.476,1.6,4.3,0.365,8.1,16.1,0.506,0.514,9.2,10.2,0.9,1.3,6.3,7.6,2.8,1.4,1.0,3.3,2.1,30.1,26.2,0.607,0.21,0.504,3.8,17.9,11.0,13.5,1.8,1.9,11.7,32.0,11.1,5.0,16.1,0.238,6.3,0.8,7.1,7.5,10,50,32,82,0.61,51,0.588,31,0.645,Finalist,Kevin Durant,1
ORL,1994,Shaquille O'Neal,21,4,3.0,289.0,1010.0,0.286,C,81,81,39.8,11.8,19.6,0.599,0.0,0.0,0.0,11.8,19.6,0.6,0.599,5.8,10.5,0.554,4.7,8.5,13.2,2.4,0.9,2.9,2.7,3.5,29.3,28.5,0.605,0.001,0.534,13.5,23.7,18.7,11.0,1.2,4.5,10.2,29.0,12.1,4.8,16.9,0.252,6.2,0.6,6.8,7.2,10,50,32,82,0.61,12,0.5,39,0.744,Contenders,Shaquille O'Neal,1
CLE,2006,LeBron James,21,2,16.0,688.0,1250.0,0.55,F,79,79,42.5,11.1,23.1,0.48,1.6,4.8,0.335,9.5,18.3,0.518,0.515,7.6,10.3,0.738,0.9,6.1,7.0,6.6,1.6,0.8,3.3,2.3,31.4,28.1,0.568,0.208,0.447,2.6,17.1,9.8,32.8,2.0,1.5,10.7,33.6,12.0,4.3,16.3,0.232,7.5,1.6,9.1,9.4,6,50,32,82,0.61,52,0.596,30,0.633,Finalist,LeBron James,1
PHI,1997,Allen Iverson,21,18,0.0,1.0,1150.0,0.001,G,76,74,40.1,8.2,19.8,0.416,2.0,6.0,0.341,6.2,13.8,0.448,0.467,5.0,7.2,0.702,1.5,2.6,4.1,7.5,2.1,0.3,4.4,3.1,23.5,18.0,0.513,0.303,0.362,4.0,7.2,5.6,33.6,2.6,0.6,16.2,28.9,3.1,1.0,4.1,0.065,2.5,-1.6,0.9,2.2,25,22,60,82,0.268,10,0.4,45,0.2,Other,Allen Iverson,1
LAL,2000,Kobe Bryant,21,12,0.0,3.0,1210.0,0.002,G,66,62,38.2,8.4,17.9,0.468,0.7,2.2,0.319,7.7,15.7,0.489,0.488,5.0,6.1,0.821,1.6,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5,21.7,0.546,0.122,0.341,4.6,12.9,8.8,22.4,2.2,1.7,11.8,26.8,6.1,4.5,10.6,0.202,3.5,1.6,5.1,4.5,1,67,15,82,0.817,9,0.889,41,0.829,Other,Kobe Bryant,1


In [0]:
# Prikaz specifinih igrača sa njihovim brojem pojavljivanj među najboljim igračima
display(table.filter(table.Rank <= 5).groupBy("Player").count().filter(col("count")>3).orderBy("count"))


Player,count
Dwight Howard,4
Hakeem Olajuwon,4
Stephen Curry,4
Russell Westbrook,4
Patrick Ewing,5
David Robinson,5
Kevin Garnett,5
Chris Paul,5
James Harden,6
Kevin Durant,6


In [0]:
# Prosečan procenat pobeda po timovima
teams_standings = sqlContext.sql("select * from `team_standings` ")
display(teams_standings.groupBy("Team").agg(mean("Win%").alias("Win%")).orderBy("Win%"))

Team,Win%
Charlotte Bobcats,0.38755555555555565
Minnesota Timberwolves,0.3939999999999999
New Jersey Nets,0.421304347826087
Washington Wizards,0.42354166666666665
Sacramento Kings,0.4315312499999999
New Orleans Pelicans,0.4475
Philadelphia 76ers,0.4487500000000001
Brooklyn Nets,0.456
Los Angeles Clippers,0.45709677419354844
Milwaukee Bucks,0.46825


In [0]:
# Broj igrača među timovima
display(table.filter(col("VoteType")!="Other").groupBy("Team","VoteType").count())

Team,VoteType,count
DET,Finalist,1
PHI,Contenders,3
MIA,Contenders,2
CLE,Contenders,3
CHI,Finalist,4
SAS,Finalist,9
CHI,Winners,5
UTA,Winners,2
LAL,Finalist,8
CHH,Contenders,1


In [0]:
#display(table.filter(col("VoteType")!="Other").groupBy("Team","VoteType").count())

## Hipoteza 1: Vrhunac karijere kod igrača je oko 27 godina

In [0]:
display(table.groupBy("Rank").agg(round(mean("Age"),2)).orderBy("Rank"))

Rank,"round(avg(Age), 2)"
1,27.81
2,27.41
3,27.28
4,27.03
5,27.13
6,27.06
7,27.94
8,27.94
9,26.66
10,28.47


In [0]:
table.select(mean("Age").alias("MeanAge")).display()
#display(table.groupBy("Year").mean("Age"))

MeanAge
27.471875


In [0]:
display(table.groupBy("Age").count().orderBy("Age"))

Age,count
19,1
20,3
21,8
22,13
23,22
24,24
25,30
26,29
27,33
28,34


In [0]:
#display(table.groupBy("Player").agg(max("VORP"),col("Age")).orderBy("Age"))*/

prime_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(PtsWon) PtsWon from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.PtsWon = b.PtsWon order by a.Age')
display(prime_table.groupBy("Age").count().orderBy("Age"))



Age,count
20,1
21,2
22,5
23,6
24,14
25,18
26,15
27,17
28,17
29,16


In [0]:
prime_pts_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(PTS) PTS from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.PTS = b.PTS order by a.Age')
#display(prime_pts_table.groupBy("Age").count().orderBy("Age"))
prime_pts_table = prime_pts_table.groupBy("Age").count().withColumnRenamed("count","PTS")

prime_usg_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(`USG%`) `USG%` from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.`USG%` = b.`USG%` order by a.Age')
#display(prime_usg_table.groupBy("Age").count().orderBy("Age"))
prime_usg_table = prime_usg_table.groupBy("Age").count().withColumnRenamed("count","USG%")

prime_vorp_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(`VORP`) `VORP` from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.`VORP` = b.`VORP` order by a.Age').withColumnRenamed("count","VORP")
prime_vorp_table = prime_vorp_table.groupBy("Age").count().withColumnRenamed("count","VORP")

prime_ws_table = sqlContext.sql('select * from `full_player_stats_with_teams` as a inner join ( select Player, max(`WS`) `WS` from `full_player_stats_with_teams` group by Player) as b where a.Player = b.Player and a.`WS` = b.`WS` order by a.Age').withColumnRenamed("count","VORP")
prime_ws_table = prime_ws_table.groupBy("Age").count().withColumnRenamed("count","WS")


prime_table = prime_pts_table.join(prime_usg_table,["Age"],"left").join(prime_vorp_table,["Age"],"left").join(prime_ws_table,["Age"],"left")
display(prime_table.orderBy("Age"))


Age,PTS,USG%,VORP,WS
20,1,1,1,1
21,3,2,3,3
22,4,2,3,4
23,5,6,6,8
24,12,11,17,17
25,20,21,19,19
26,17,15,14,13
27,18,14,20,22
28,13,16,11,8
29,15,17,15,12


Većina plotova liče na normalnu distribuciju sa 26-27 kao centar.

Mada postoji neki pik kod 25-te godine, mada to može da se gleda kao nesavršen gde igrači još nisu došli do svog vrhunca ( npr. Nikola Jokic)

Tako da mogu da kažem da je hipoteza tačan

## Hipoteza 2: Sezona 2015 je bila prekretnica

In [0]:
# Stef Kari je 2015te godine osvojio MVP nagradu, i pokazao NBA ligi moć trojki
display(table.filter(table.Year==2015).filter(table.Rank==1))

Team,Year,Player,Age,Rank,FirstPlace,PtsWon,PtsMax,%,Pos,G,Gs,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Rk,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post,VoteType
GSW,2015,Stephen Curry,26,1,100.0,1198.0,1300.0,0.922,G,80,80,32.7,8.2,16.8,0.487,3.6,8.1,0.443,4.6,8.7,0.528,0.594,3.9,4.2,0.914,0.7,3.6,4.3,7.7,2.0,0.2,3.1,2.0,23.8,28.0,0.638,0.482,0.251,2.4,11.4,7.0,38.6,3.0,0.5,14.3,28.9,11.5,4.1,15.7,0.288,8.2,1.7,9.9,7.9,1,67,15,82,0.817,51,0.824,31,0.806,Winners


In [0]:
display(table.groupBy("Year").agg(mean("3PA")).orderBy("Year"))

Year,avg(3PA)
1990,1.33999999538064
1991,2.009999995678663
1992,1.869999995827675
1993,1.4800000295042992
1994,1.1300000198185445
1995,1.7300000019371509
1996,1.6899999924004079
1997,2.209999991953373
1998,1.6900000303983689
1999,1.369999984651804


In [0]:
display(table.groupBy("Year").agg(mean("3PAR")).orderBy("Year"))

Year,avg(3PAR)
1990,0.0784999999217689
1991,0.13139999927952886
1992,0.11610000063665211
1993,0.08119999910704792
1994,0.07300000035902485
1995,0.11739999963901937
1996,0.10160000089090318
1997,0.11900000264868141
1998,0.10289999842643738
1999,0.07579999847803265


In [0]:
mega_stat_table = table.select(col("Player"),col("Year"),(col("PTS")+col("AST")+col("TRB")).alias("Total"))
display(mega_stat_table.groupBy("Year").mean("Total").orderBy("Year"))

Year,avg(Total)
1990,40.599999237060544
1991,38.079999923706055
1992,36.38000049591064
1993,39.20999946594238
1994,37.07999954223633
1995,37.179999923706056
1996,36.90000057220459
1997,36.35000076293945
1998,34.970000648498534
1999,35.77000007629395


In [0]:
display(table.filter(table.Rank <= 5).groupBy("Year").agg(mean("OBPM"),mean("DBPM")).orderBy("Year"))

Year,avg(OBPM),avg(DBPM)
1990,6.840000104904175,1.4399999618530273
1991,6.839999961853027,2.0
1992,5.3199999809265135,2.359999990463257
1993,5.479999995231628,1.5799999475479125
1994,4.960000038146973,2.7200000047683717
1995,4.1,1.8199999719858169
1996,5.540000009536743,2.0
1997,5.880000019073487,0.7800000429153442
1998,5.019999980926514,0.9199999958276749
1999,4.000000047683716,1.4200000047683716


Od 2015te počinje vidni rast kod šutiranja trojki, kao i Ofanzivni rejting ( mada nema vidljive promene kod Defanzivnog rejtinga).

Takođe vidi se da se očekuje dosta više od igrača u zadnjih par godina

## ML umesto hipoteze

In [0]:
display(table.filter(table.Rank <= 10).groupBy("Rank").agg(mean("FTA"),10*mean("TS%")).orderBy("Rank"))

#display(table.filter(table.Rank <= 10).groupBy("Rank").agg(mean("FTA"),mean("TS%")).orderBy("Rank"))

Rank,avg(FTA),(avg(TS%) * 10)
1,7.978124983608723,5.960000045597553
2,8.178124971687794,5.909062502905726
3,7.8968749940395355,5.807500006631017
4,8.153124958276749,5.7359375432133675
5,7.403125032782555,5.771874990314245
6,6.7749999687075615,5.713750086724758
7,6.768750011920929,5.6809375155717134
8,6.465624958276749,5.611562477424741
9,5.903125010430813,5.659687481820583
10,6.043749950826168,5.660625034943223


In [0]:
from pyspark.ml.feature import VectorAssembler,Normalizer
from pyspark.ml.regression import LinearRegression
import seaborn as sns

In [0]:
import six
result = []
for i in table.columns:
    if not( isinstance(table.select(i).take(1)[0][0], six.string_types)):
        #print("Corrrelation to Points% ", i,table.stat.corr('%',i))
        result.append((i,table.stat.corr('%',i)))

        
#sor = sorted(result, key=lambda tup: abs(tup[1]))
for s in result:
    print(s)

('Year', 0.0031281381695306126)
('Age', 0.015255754288700681)
('Rank', -0.8843668629941412)
('FirstPlace', 0.8270298306729735)
('PtsWon', 0.9908893423206331)
('PtsMax', 0.001175398265898381)
('%', 1.0)
('G', 0.13811387067595837)
('Gs', 0.15369208481783725)
('MP', 0.10485729041155048)
('FG', 0.3663153766801363)
('FGA', 0.2685188632552057)
('FG%', 0.16873998060773324)
('3P', 0.10824297511312117)
('3PA', 0.09964992805959161)
('3P%', 0.0049891310393228964)
('2P', 0.25460610640523296)
('2PA', 0.1787105191419151)
('2P%', 0.22749065915873395)
('eFG%', 0.22899808449345632)
('FT', 0.30139677580256385)
('FTA', 0.2954275078992277)
('FT%', 0.03709133339315942)
('ORB', 0.044988250027316115)
('DRB', 0.1446843329086436)
('TRB', 0.11760985949638554)
('AST', 0.08445258224025995)
('STL', 0.08870966212482692)
('BLK', 0.08374534609262627)
('TOV', 0.22831936654047444)
('PF', -0.04424282223984683)
('PTS', 0.3918632850653542)
('PER', 0.5887733686630685)
('TS%', 0.2571957082003864)
('3PAr', 0.0470716649160999

Iznad vidimo korelaciju svih kolona sa % kolonom( procenat osvojenih glasova za nagradu )

In [0]:
df = table.select(col("Rk"),col("Win%"),col("PTS"),col("OWS"),col("OBPM"),col("PER"), \
                  col("WS"),col("WS/48"),col("BPM"),col("VORP"),col("%"))
#df = df.withColumn("Pos",when(col("Pos")=="G",lit(0)).when(col("Pos")=="F",lit(1)).otherwise(lit(2)))
#display(df)

df = df.toPandas()
sns.set(style="ticks")
g = sns.pairplot(df)

Ćelija iznad je pokretana ali je rezultat 4k slika koju Databricks nije mogao da formatira, tako da sam ovde obrisao rezultat a sliku sačuvao
Ovde vidimo par linernih veza između nekih kolona, tako da ću njih izbaciti zarad boljeg učenja modela
Kolone sa velikom linearnom vezom su:
  * Rk - Win%
  * WS - OWS
  * BPM - VORP
  * VORP - WS
  * WS - WS/48

In [0]:
print(df.columns)

['Rk', 'Win%', 'PTS', 'OWS', 'OBPM', 'PER', 'WS', 'WS/48', 'BPM', 'VORP', '%']


In [0]:
X = table.select(col("Win%"),col("PTS"),col("OWS"),col("OBPM"),col("PER"), \
                  col("VORP"),col("%"))
#X = X.withColumn("Pos",when(col("Pos")=="G",lit(0)).when(col("Pos")=="F",lit(1)).otherwise(lit(2)))
#X = X.withColumn("Rk",lit(30)-col("Rk")) # 30 je max

print(X.columns)
#Y = table.select(col("Player"),col("Year"),)

vectorAssembler = VectorAssembler(inputCols=['Win%', 'PTS', 'OWS', 'OBPM', 'PER', 'VORP'],outputCol='features')
vector_X = vectorAssembler.transform(X)

normilizer = Normalizer(inputCol='features', outputCol='normFeatures',p=1.0)
vector_X = normilizer.transform(vector_X)

vector_X = vector_X.select(['normFeatures','%'])
vector_X.show(3)


['Win%', 'PTS', 'OWS', 'OBPM', 'PER', 'VORP', '%']
+--------------------+-----+
|        normFeatures|    %|
+--------------------+-----+
|[0.01127361371573...|0.701|
|[0.01203062323772...|0.655|
|[0.01332373025565...|0.627|
+--------------------+-----+
only showing top 3 rows



In [0]:
splits = vector_X.randomSplit([0.7,0.3])
tran_vector = splits[0]
test_vector = splits[1]

In [0]:
lr = LinearRegression(featuresCol='normFeatures',labelCol='%',regParam=0.1,elasticNetParam=0.0)
lr_model = lr.fit(tran_vector)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))




Coefficients: [-7.713846792797259,-0.3079457209987156,0.5565734419312252,1.4773326978678527,-1.1295820910508945,3.7078829575627466]
Intercept: 0.37736346527508896


In [0]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features, current: normFeatures)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: %)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max n

In [0]:
trainingSummary = lr_model.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

numIterations: 0
objectiveHistory: [0.0]
+--------------------+
|           residuals|
+--------------------+
| 0.08894220971360195|
|-0.24391582962914587|
| -0.1301865552196248|
| -0.2450873410092732|
|   0.445478708100525|
| -0.2640397677621344|
|-0.06722549631776667|
|-0.26782654691911884|
|-0.14052218677074238|
| -0.1439411738546177|
| 0.12767774760485107|
| 0.49438447267782404|
|-0.05474751256269633|
| -0.0791471558270474|
|  0.4113291394126025|
|-0.10095915907020886|
| 0.04514299234464331|
|-0.03484003912160488|
|-0.29813938254462713|
|-0.04628235489237076|
+--------------------+
only showing top 20 rows

RMSE: 0.247604
r2: 0.283800


In [0]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.247604
r2: 0.283800


In [0]:
tran_vector.describe().show()

+-------+-------------------+
|summary|                  %|
+-------+-------------------+
|  count|                221|
|   mean|0.24532579335466245|
| stddev|0.29324087816523714|
|    min|              0.001|
|    max|              0.998|
+-------+-------------------+



In [0]:
test_result = lr_model.evaluate(test_vector)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 0.302989


In [0]:
predictions = lr_model.transform(test_vector)
predictions.select("prediction","%","normFeatures").show()

+--------------------+-----+--------------------+
|          prediction|    %|        normFeatures|
+--------------------+-----+--------------------+
|  0.1664023433871121| 0.27|[0.00890868594178...|
|  0.1360090173649113|0.701|[0.01127361371573...|
|-0.02411836433039...|0.655|[0.01203062323772...|
| -0.1809599524511309|0.043|[0.01240238859119...|
|0.007960224022718732|0.627|[0.01332373025565...|
|-0.01721923858037333| 0.05|[0.01344424053436...|
|  0.3139696630287982|0.466|[0.00805586008283...|
| 0.15282877275095524|0.023|[0.00808342598678...|
| 0.35147692972608546|0.043|[0.00822484228448...|
| 0.31650892810991976|0.122|[0.00890124745695...|
| 0.27103311835712773|0.904|[0.00920696110442...|
| 0.21724555126668657|0.009|[0.00939697046738...|
| 0.12392664939338821|0.268|[0.00998068277974...|
| 0.24651477794678295|0.078|[0.01047298780888...|
|  0.3365200163062551|0.061|[0.01149543432031...|
| 0.19083987637415128|0.569|[0.01155096639273...|
|  0.1353182780454053| 0.02|[0.01196503355236...|


## Data streaming

In [0]:
from pyspark.sql.types import *
files_path = "/FileStore/tables/mvp_voting/"

staticInputDF = (spark.read.option("header",True).csv(files_path))

schema = staticInputDF.schema

display(staticInputDF)

Player,Year,Age,Team,FirstPlace,PtsWon,PtsMax,%,GP,MP,PTS,TRP,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
Karl Malone,1999,35,UTA,44.0,827.0,1180,0.701,49,37.4,23.8,9.4,4.1,1.3,0.6,.493,.000,.788,9.6,.252
Alonzo Mourning,1999,28,MIA,36.0,773.0,1180,0.655,46,38.1,20.1,11.0,1.6,0.7,3.9,.511,.000,.652,7.9,.216
Tim Duncan,1999,22,SAS,30.0,740.0,1180,0.627,50,39.3,21.7,11.4,2.4,0.9,2.5,.495,.143,.690,8.7,.213
Allen Iverson,1999,23,PHI,5.0,319.0,1180,0.270,48,41.5,26.8,4.9,4.6,2.3,0.1,.412,.291,.751,7.2,.173
Jason Kidd,1999,25,PHO,2.0,159.0,1180,0.135,50,41.2,16.9,6.8,10.8,2.3,0.4,.444,.366,.757,8.1,.188
Shaquille O'Neal,1999,26,LAL,1.0,89.0,1180,0.075,49,34.8,26.3,10.7,2.3,0.7,1.7,.576,.000,.540,9.0,.255
Chris Webber,1999,25,SAC,0.0,51.0,1180,0.043,42,40.9,20.0,13.0,4.1,1.4,2.1,.486,.118,.454,3.6,.102
Grant Hill,1999,26,DET,0.0,39.0,1180,0.033,50,37.0,21.1,7.1,6.0,1.6,0.5,.479,.000,.752,7.3,.189
Gary Payton,1999,30,SEA,0.0,35.0,1180,0.030,50,40.2,21.7,4.9,8.7,2.2,0.2,.434,.295,.721,7.2,.172
Kevin Garnett,1999,22,MIN,0.0,9.0,1180,0.008,47,37.9,20.8,10.4,4.3,1.7,1.8,.460,.286,.704,5.4,.146


In [0]:
from pyspark.sql.functions import *
from time import sleep

streamingInputDF = spark.readStream.schema(schema).option("header",True).option("maxFilesPerTrigger", 1).csv(files_path)



streamingCountsDF = streamingInputDF.filter(col("FirstPlace")>0).groupBy(col("Team")).count()

print(streamingCountsDF.isStreaming)

True


In [0]:
query = streamingCountsDF.writeStream.format("memory").queryName("counts").outputMode("complete").start()

In [0]:
sleep(35)

In [0]:
%sql
select * from counts order by count desc

Team,count
LAL,1
NYK,1
UTA,1
PHI,1
SAS,1
HOU,1
CHI,1


In [0]:
sleep(20)

In [0]:
%sql
select * from counts order by count desc;

Team,count
SAS,2
LAL,2
PHI,2
CHI,2
NYK,1
POR,1
UTA,1
HOU,1


In [0]:
sleep(20)

In [0]:
%sql
select * from counts order by count desc;

Team,count
CHI,5
SAS,3
POR,2
NYK,2
LAL,2
UTA,2
PHI,2
HOU,2
PHO,1


In [0]:
query.stop()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-2903055546864264> in <module>
----> 1 query.stop()

/databricks/spark/python/pyspark/sql/streaming.py in stop(self)
    155         """Stop this streaming query.
    156         """
--> 157         self._jsq.stop()
    158 
    159     def explain(self, extended=False):

/databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__(self, *args)
   1302 
   1303         answer = self.gateway_client.send_command(command)
-> 1304         return_value = get_return_value(
   1305             answer, self.gateway_client, self.target_id, self.name)
   1306 

/databricks/spark/python/pyspark/sql/utils.py in deco(*a, **kw)
    115     def deco(*a, **kw):
    116         try:
--> 117             return f(*a, **kw)
    118         except py4j.protocol.Py4JJavaError as e:
    119             converted = convert_exceptio